In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import  OrdinalEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve

import lightgbm as lgb
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import optuna

numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

train = pd.read_csv('./data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./data/test.csv').drop(columns=['ID'])

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

# 수치형 컬럼들을 0으로 채움
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

# **PCA**로 차원 축소 후 군집화 수행 (군집 수는 3~6 사이로 실험)
pca = PCA(n_components=7, random_state=42)
X_pca = pca.fit_transform(X_train_encoded[numeric_columns])

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans_clusters = kmeans.fit_predict(X_pca)

# 새로운 군집 피처 추가
X_train_encoded['cluster_group'] = kmeans_clusters
X_test_encoded['cluster_group'] = kmeans.predict(pca.transform(X_test_encoded[numeric_columns]))

from imblearn.over_sampling import ADASYN

# ADASYN 적용
adasyn = ADASYN(sampling_strategy=0.5, random_state=42)
X_train_augmented, y_train_augmented = adasyn.fit_resample(X_train_encoded, y)
print("ADASYN 적용 후 데이터 크기:", X_train_augmented.shape, y_train_augmented.shape)


# 새로운 파생 변수 생성
X_train_augmented["시도_대비_이식_비율"] = (
    X_train_augmented["임신 시도 또는 마지막 임신 경과 연수"] / (X_train_augmented["배아 이식 경과일"] + 1)
)
X_train_augmented["난자_활용도"] = (
    X_train_augmented["수집된 신선 난자 수"] / (X_train_augmented["총 생성 배아 수"] + 1)
)

# 테스트 데이터에도 동일하게 적용
X_test_encoded["시도_대비_이식_비율"] = (
    X_test_encoded["임신 시도 또는 마지막 임신 경과 연수"] / (X_test_encoded["배아 이식 경과일"] + 1)
)
X_test_encoded["난자_활용도"] = (
    X_test_encoded["수집된 신선 난자 수"] / (X_test_encoded["총 생성 배아 수"] + 1)
)

print("새로운 변수 추가 완료!")

# 스케일링 적용
scaler = StandardScaler()
X_train_encoded[numeric_columns] = scaler.fit_transform(X_train_encoded[numeric_columns])
X_test_encoded[numeric_columns] = scaler.transform(X_test_encoded[numeric_columns])

print("-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------")
lgb_est = LGBMClassifier(
    class_weight='balanced',
    random_state=42,
    objective='binary',
    n_estimators=1000,
    subsample=0.82,
    colsample_bytree=0.85,
    learning_rate=0.01,
    num_leaves=49,
    max_depth=17,
    device='gpu',  # GPU 사용
    gpu_platform_id=0,
    gpu_device_id=2
)

# LightGBM 모델 학습
lgb_est.fit(X_train_encoded, y)

# LightGBM으로 Feature Importance 확인
feature_importance = pd.DataFrame({
    'feature': X_train_encoded.columns,
    'importance': lgb_est.feature_importances_
}).sort_values(by='importance', ascending=False)

low_importance_features = feature_importance[feature_importance['importance'] <= 10]['feature'].tolist()
print("중요도가 낮은 피처:", low_importance_features)

# 중요도가 낮은 피처 제거
X_train_encoded = X_train_encoded.drop(columns=low_importance_features)
X_test_encoded = X_test_encoded.drop(columns=low_importance_features)

X_train, X_val, y_train, y_val = train_test_split(X_train_encoded, y, test_size=0.1, random_state=42)

2025-02-13 01:19:52.017691: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-13 01:19:52.032227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739377192.050344 2865094 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739377192.056026 2865094 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 01:19:52.074188: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

ADASYN 적용 후 데이터 크기: (285026, 68) (285026,)
새로운 변수 추가 완료!
-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 743
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 63
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.89 MB) transferred to GPU in 0.007982 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 

In [2]:
def objective_lgb(trial):
    params = {
        'objective': 'binary',
        'n_estimators': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 31, 100),
        'max_depth': trial.suggest_int('max_depth', -1, 20),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 2
    }
    
    model = LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, y_pred)

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=200)
print("Best parameters for LightGBM:", study_lgb.best_params)


[I 2025-02-13 01:20:24,293] A new study created in memory with name: no-name-2bebfdc1-87c1-4087-afd8-311dfee12bbf


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 59727, number of negative: 170988
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 736
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 62
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.40 MB) transferred to GPU in 0.003813 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258878 -> initscore=-1.051809
[LightGBM] [Info] Start training from score -1.051809


[I 2025-02-13 01:20:32,842] Trial 0 finished with value: 0.7410336260301575 and parameters: {'learning_rate': 0.012382544477432275, 'num_leaves': 58, 'max_depth': 15, 'subsample': 0.8096413438602101, 'colsample_bytree': 0.5651157365930761}. Best is trial 0 with value: 0.7410336260301575.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 59727, number of negative: 170988
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 736
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 62
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.40 MB) transferred to GPU in 0.003800 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258878 -> initscore=-1.051809
[LightGBM] [Info] Start training from score -1.051809


[I 2025-02-13 01:20:44,274] Trial 1 finished with value: 0.7386364791941519 and parameters: {'learning_rate': 0.0016595165670645349, 'num_leaves': 78, 'max_depth': 14, 'subsample': 0.7833981621212638, 'colsample_bytree': 0.6778904334661485}. Best is trial 0 with value: 0.7410336260301575.


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 59727, number of negative: 170988
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 736
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 62
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.40 MB) transferred to GPU in 0.003882 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258878 -> initscore=-1.051809
[LightGBM] [Info] Start training from score -1.051809
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

[I 2025-02-13 01:42:03,359] Trial 198 finished with value: 0.741621925705627 and parameters: {'learning_rate': 0.01404301118940053, 'num_leaves': 33, 'max_depth': 7, 'subsample': 0.9588693733821373, 'colsample_bytree': 0.5190734664368356}. Best is trial 198 with value: 0.741621925705627.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 59727, number of negative: 170988
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 736
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 62
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.40 MB) transferred to GPU in 0.004736 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258878 -> initscore=-1.051809
[LightGBM] [Info] Start training from score -1.051809
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

[I 2025-02-13 01:42:09,444] Trial 199 finished with value: 0.7415015928686496 and parameters: {'learning_rate': 0.011915846698069063, 'num_leaves': 34, 'max_depth': 6, 'subsample': 0.9560947979070115, 'colsample_bytree': 0.5234945032254626}. Best is trial 198 with value: 0.741621925705627.


Best parameters for LightGBM: {'learning_rate': 0.01404301118940053, 'num_leaves': 33, 'max_depth': 7, 'subsample': 0.9588693733821373, 'colsample_bytree': 0.5190734664368356}


In [3]:
def objective_xgb(trial):
    params = {
        'n_estimators': 300,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'tree_method': 'hist',  # 권장 방식
        'device': 'cuda'
    }
    
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, y_pred)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=1000)
print("Best parameters for XGBoost:", study_xgb.best_params)

[I 2025-02-13 01:42:09,455] A new study created in memory with name: no-name-7462bcc2-905c-420c-93cf-d47462e93587
/home/najo/.conda/envs/dip/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [01:42:10] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2025-02-13 01:42:10,677] Trial 0 finished with value: 0.7413913849036189 and parameters: {'learning_rate': 0.0425655090900175, 'max_depth': 7, 'subsample': 0.537328696190243, 'colsample_bytree': 0.6141381469311394, 'gamma': 4.486291713428394}. Best is trial 0 with value: 0.7413913849036189.
[I 2025-02-13 01:42

Best parameters for XGBoost: {'learning_rate': 0.07820838231358765, 'max_depth': 4, 'subsample': 0.6162659283584726, 'colsample_bytree': 0.9210692408000515, 'gamma': 2.8703958280747597}


In [5]:
def objective_cat(trial):
    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'depth': trial.suggest_int('depth', 4, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'random_seed': 42,
        'task_type': 'GPU',
        'devices': '0'
    }
    
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, verbose=0)
    y_pred = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, y_pred)

study_cat = optuna.create_study(direction='maximize')
study_cat.optimize(objective_cat, n_trials=200)
print("Best parameters for CatBoost:", study_cat.best_params)

[I 2025-02-13 02:03:42,335] A new study created in memory with name: no-name-9f333fef-bb2b-49a0-a1f1-583ba871f2c1
[I 2025-02-13 02:03:47,392] Trial 0 finished with value: 0.7410498925473346 and parameters: {'learning_rate': 0.015988720233911444, 'depth': 7, 'l2_leaf_reg': 1.9575276656834564}. Best is trial 0 with value: 0.7410498925473346.
[I 2025-02-13 02:03:50,981] Trial 1 finished with value: 0.7405269202016599 and parameters: {'learning_rate': 0.012977779580679003, 'depth': 5, 'l2_leaf_reg': 0.009285330945143931}. Best is trial 0 with value: 0.7410498925473346.
[I 2025-02-13 02:03:54,323] Trial 2 finished with value: 0.7395377334764723 and parameters: {'learning_rate': 0.00951281812981531, 'depth': 4, 'l2_leaf_reg': 0.1276346065459466}. Best is trial 0 with value: 0.7410498925473346.
[I 2025-02-13 02:04:03,870] Trial 3 finished with value: 0.7359897998848602 and parameters: {'learning_rate': 0.0010302909753974001, 'depth': 9, 'l2_leaf_reg': 9.702175674651553}. Best is trial 0 with 

Best parameters for CatBoost: {'learning_rate': 0.04136652091082273, 'depth': 5, 'l2_leaf_reg': 4.525500424821638}


In [7]:
print("-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for LightGBM: {'learning_rate': 0.010682507488382762, 'num_leaves': 49, 'max_depth': 17, 'subsample': 0.8215664737448534, 'colsample_bytree': 0.8497893445033722}
# Best parameters for LightGBM: {'learning_rate': 0.010133602189986561, 'num_leaves': 35, 'max_depth': 9, 'subsample': 0.9394014624416219, 'colsample_bytree': 0.531614077286386}
# Best parameters for LightGBM: {'learning_rate': 0.01404301118940053, 'num_leaves': 33, 'max_depth': 7, 'subsample': 0.9588693733821373, 'colsample_bytree': 0.5190734664368356}
lgb_est = LGBMClassifier(
    class_weight='balanced',
    random_state=42,
    objective='binary',
    n_estimators=1000,
    subsample=0.959,
    colsample_bytree=0.519,
    learning_rate=0.014,
    num_leaves=33,
    max_depth=7,
    device='gpu',  # GPU 사용
    gpu_platform_id=0,
    gpu_device_id=2
)

# LightGBM 모델 학습
lgb_est.fit(X_train, y_train)
y_train_pred_lgb = lgb_est.predict_proba(X_val)[:, 1]
roc_auc_lgb = roc_auc_score(y_val, y_train_pred_lgb)

print("-------------------------- 2. XGBoost 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for XGBoost: {'learning_rate': 0.027828379673739464, 'max_depth': 9, 'subsample': 0.8401352803766647, 'colsample_bytree': 0.817257623227272, 'gamma': 4.054300711621877}
# Best parameters for XGBoost: {'learning_rate': 0.0815351146687336, 'max_depth': 4, 'subsample': 0.9118736980020461, 'colsample_bytree': 0.9163782063094905, 'gamma': 0.9029812557206023}
# Best parameters for XGBoost: {'learning_rate': 0.07820838231358765, 'max_depth': 4, 'subsample': 0.6162659283584726, 'colsample_bytree': 0.9210692408000515, 'gamma': 2.8703958280747597}
xgb_est = XGBClassifier(
    eval_metric='auc',
    random_state=42,
    n_estimators=300,
    learning_rate=0.078,
    max_depth=4,
    subsample=0.616,
    colsample_bytree=0.921,
    gamma=2.870,
    tree_method='hist',  # 새 권장 방식
    device='cuda'        # GPU 사용
)

# XGBoost 모델 학습
xgb_est.fit(X_train, y_train)
y_train_pred_xgb = xgb_est.predict_proba(X_val)[:, 1]
roc_auc_xgb = roc_auc_score(y_val, y_train_pred_xgb)

print("-------------------------- 3. CatBoost 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for CatBoost: {'learning_rate': 0.03222538120048102, 'depth': 6, 'l2_leaf_reg': 1.380048205342132}
# Best parameters for CatBoost: {'learning_rate': 0.023272410542836165, 'depth': 6, 'l2_leaf_reg': 0.02493607891017943}
# Best parameters for CatBoost: {'learning_rate': 0.04136652091082273, 'depth': 5, 'l2_leaf_reg': 4.525500424821638}
catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.0413,
    depth=5,
    l2_leaf_reg=4.5255,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=100,  # 경고 메시지 최소화
    task_type='GPU',
    devices='1'
)

# CatBoost 모델 학습
catboost_model.fit(X_train, y_train)
y_train_pred_cat = catboost_model.predict_proba(X_val)[:, 1]
roc_auc_cat = roc_auc_score(y_val, y_train_pred_cat)

# -------------------------- 2. TabNetClassifier 적용 --------------------------
print("-------------------------- 2. TabNetClassifier 학습 --------------------------")

tabnet_clf = TabNetClassifier(
    n_d=256, n_a=256,
    n_steps=5,
    gamma=1.5,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    scheduler_params={"step_size": 10, "gamma": 0.9},
    seed=42
)

tabnet_clf.fit(
    X_train.values, y_train.values,
    eval_set=[(X_val.values, y_val.values)],
    eval_metric=['auc'],
    patience=30,
    max_epochs=200
)
y_pred_tabnet = tabnet_clf.predict_proba(X_val.values)[:, 1]
roc_auc_tabnet = roc_auc_score(y_val, y_pred_tabnet)

# -------------------------- 4. Weighted Ensemble --------------------------
print("-------------------------- 4. Weighted Ensemble 최적 가중치 탐색 --------------------------")

def objective_weights(trial):
    w1 = trial.suggest_float('w1', 0, 1)
    w2 = trial.suggest_float('w2', 0, 1 - w1)
    remaining = max(1 - w1 - w2, 0)  # 음수 방지
    w3 = trial.suggest_float('w3', 0, remaining)
    w4 = max(remaining - w3, 0)  # 마지막 가중치는 남은 값으로 설정

    final_pred_proba = (
        w1 * y_train_pred_lgb +
        w2 * y_train_pred_xgb +
        w3 * y_pred_tabnet +
        w4 * y_train_pred_cat
    )
    return roc_auc_score(y_val, final_pred_proba)

study_weights = optuna.create_study(direction='maximize')
study_weights.optimize(objective_weights, n_trials=1000)
best_weights = study_weights.best_params

print(f"최적 가중치: LightGBM={best_weights['w1']:.2f}, XGBoost={best_weights['w2']:.2f}, TabNetClassifier={best_weights['w3']:.2f}, CatBoost={1 - best_weights['w1'] - best_weights['w2'] - best_weights['w3']:.2f}")
# 최적 가중치: LightGBM=0.21, XGBoost=0.36, TabNetClassifier=0.28, CatBoost=0.15

# 최종 앙상블 적용
final_pred_proba = (
    best_weights['w1'] * y_train_pred_lgb +
    best_weights['w2'] * y_train_pred_xgb +
    best_weights['w3'] * y_pred_tabnet +
    (1 - best_weights['w1'] - best_weights['w2'] - best_weights['w3']) * y_train_pred_cat
)
roc_auc_final = roc_auc_score(y_val, final_pred_proba)

print(f"LightGBM 모델의 ROC-AUC 점수: {roc_auc_lgb:.4f}")
print(f"XGBoost 모델의 ROC-AUC 점수: {roc_auc_xgb:.4f}")
print(f"CatBoost 모델의 ROC-AUC 점수: {roc_auc_cat:.4f}")
print(f"TabNetClassifier 모델의 ROC-AUC 점수: {roc_auc_tabnet:.4f}")

print(f"Final 앙상블 모델의 ROC-AUC 점수: {roc_auc_final:.4f}")

-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 59727, number of negative: 170988
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 732
[LightGBM] [Info] Number of data points in the train set: 230715, number of used features: 62
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.40 MB) transferred to GPU in 0.003825 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with po

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 3.65ms	remaining: 3.65s
100:	total: 284ms	remaining: 2.53s
200:	total: 547ms	remaining: 2.17s
300:	total: 811ms	remaining: 1.88s
400:	total: 1.07s	remaining: 1.6s
500:	total: 1.34s	remaining: 1.33s
600:	total: 1.6s	remaining: 1.06s
700:	total: 1.87s	remaining: 797ms
800:	total: 2.13s	remaining: 530ms
900:	total: 2.4s	remaining: 263ms
999:	total: 2.67s	remaining: 0us
-------------------------- 2. TabNetClassifier 학습 --------------------------


/home/najo/.conda/envs/dip/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.66059 | val_0_auc: 0.70103 |  0:00:16s
epoch 1  | loss: 0.50948 | val_0_auc: 0.72038 |  0:00:32s
epoch 2  | loss: 0.50327 | val_0_auc: 0.72401 |  0:00:47s
epoch 3  | loss: 0.5061  | val_0_auc: 0.7181  |  0:01:03s
epoch 4  | loss: 0.50255 | val_0_auc: 0.72157 |  0:01:17s
epoch 5  | loss: 0.50282 | val_0_auc: 0.72068 |  0:01:32s
epoch 6  | loss: 0.49876 | val_0_auc: 0.72917 |  0:01:46s
epoch 7  | loss: 0.49682 | val_0_auc: 0.73138 |  0:01:59s
epoch 8  | loss: 0.49631 | val_0_auc: 0.72939 |  0:02:15s
epoch 9  | loss: 0.50051 | val_0_auc: 0.72822 |  0:02:31s
epoch 10 | loss: 0.49806 | val_0_auc: 0.73119 |  0:02:46s
epoch 11 | loss: 0.49738 | val_0_auc: 0.72949 |  0:03:01s
epoch 12 | loss: 0.50048 | val_0_auc: 0.72614 |  0:03:15s
epoch 13 | loss: 0.49852 | val_0_auc: 0.72652 |  0:03:28s
epoch 14 | loss: 0.49746 | val_0_auc: 0.73067 |  0:03:41s
epoch 15 | loss: 0.49629 | val_0_auc: 0.73282 |  0:03:55s
epoch 16 | loss: 0.49775 | val_0_auc: 0.73305 |  0:04:11s
epoch 17 | los

/home/najo/.conda/envs/dip/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-02-13 02:46:41,575] A new study created in memory with name: no-name-740c4f84-f245-44fd-91f6-b124178437d6
[I 2025-02-13 02:46:41,585] Trial 0 finished with value: 0.7421406776799067 and parameters: {'w1': 0.023707275279950313, 'w2': 0.66792347067769, 'w3': 0.17180173512120137}. Best is trial 0 with value: 0.7421406776799067.
[I 2025-02-13 02:46:41,592] Trial 1 finished with value: 0.7417463261767491 and parameters: {'w1': 0.5654150711005158, 'w2': 0.21203871481437614, 'w3': 0.14146564116495605}. Best is trial 0 with value: 0.7421406776799067.
[I 2025-02-13 02:46:41,599] Trial 2 finished with value: 0.7410783257923335 and parameters: {'w1': 0.9222189744301093, 'w2': 0.01724171922051562, 'w3': 0.048461942580299636}. Best is trial 0 with value: 0.7421406776799067.
[I 2025-02-13 02:46:41,606] Trial 3 fini

-------------------------- 4. Weighted Ensemble 최적 가중치 탐색 --------------------------


[I 2025-02-13 02:46:41,789] Trial 17 finished with value: 0.7421073889981027 and parameters: {'w1': 0.1391001092204217, 'w2': 0.5953643871897983, 'w3': 0.18791322403985014}. Best is trial 3 with value: 0.7422999906709695.
[I 2025-02-13 02:46:41,806] Trial 18 finished with value: 0.741665230735542 and parameters: {'w1': 0.12898422176450952, 'w2': 0.8116030254371565, 'w3': 0.0008390341265162755}. Best is trial 3 with value: 0.7422999906709695.
[I 2025-02-13 02:46:41,824] Trial 19 finished with value: 0.7420034955125594 and parameters: {'w1': 0.3795223486115724, 'w2': 0.1305068403368292, 'w3': 0.3332975623355177}. Best is trial 3 with value: 0.7422999906709695.
[I 2025-02-13 02:46:41,840] Trial 20 finished with value: 0.74153904565023 and parameters: {'w1': 0.005293094103056006, 'w2': 0.9765849474469971, 'w3': 0.0004227512968662632}. Best is trial 3 with value: 0.7422999906709695.
[I 2025-02-13 02:46:41,858] Trial 21 finished with value: 0.742128941028027 and parameters: {'w1': 0.13174849

최적 가중치: LightGBM=0.00, XGBoost=0.23, TabNetClassifier=0.22, CatBoost=0.55
LightGBM 모델의 ROC-AUC 점수: 0.7409
XGBoost 모델의 ROC-AUC 점수: 0.7415
CatBoost 모델의 ROC-AUC 점수: 0.7421
TabNetClassifier 모델의 ROC-AUC 점수: 0.7393
Final 앙상블 모델의 ROC-AUC 점수: 0.7424


In [ ]:
# LightGBM 모델의 ROC-AUC 점수: 0.7359
# XGBoost 모델의 ROC-AUC 점수: 0.7308
# CatBoost 모델의 ROC-AUC 점수: 0.7354
# StackingClassifier 모델의 ROC-AUC 점수: 0.7201

# SMOTETomek 적용 0.8
# LightGBM 모델의 ROC-AUC 점수: 0.7364 
# XGBoost 모델의 ROC-AUC 점수: 0.7320 ++++++++++++++++++++++++++++++++ 최고
# CatBoost 모델의 ROC-AUC 점수: 0.7361

# SMOTETomek 적용 0.5
# LightGBM 모델의 ROC-AUC 점수: 0.7366 ++++++++++++++++++++++++++++++++ 최고
# XGBoost 모델의 ROC-AUC 점수: 0.7317
# CatBoost 모델의 ROC-AUC 점수: 0.7363

# # 미적용
# LightGBM 모델의 ROC-AUC 점수: 0.7364
# XGBoost 모델의 ROC-AUC 점수: 0.7276
# CatBoost 모델의 ROC-AUC 점수: 0.7368
# Final 모델의 ROC-AUC 점수: 0.7359

# Q1-3 제거
# LightGBM 모델의 ROC-AUC 점수: 0.7364
# XGBoost 모델의 ROC-AUC 점수: 0.7292
# CatBoost 모델의 ROC-AUC 점수: 0.7369 ++++++++++++++++++++++++++++++++ 최고
# Final 모델의 ROC-AUC 점수: 0.7361

# # Feature engnieering 적용
# LightGBM 모델의 ROC-AUC 점수: 0.7362
# XGBoost 모델의 ROC-AUC 점수: 0.7299
# CatBoost 모델의 ROC-AUC 점수: 0.7368
# Final 모델의 ROC-AUC 점수: 0.7363

# KNMeans 적용 군집
# LightGBM 모델의 ROC-AUC 점수: 0.7363
# XGBoost 모델의 ROC-AUC 점수: 0.7301
# CatBoost 모델의 ROC-AUC 점수: 0.7368
# Final 모델의 ROC-AUC 점수: 0.7364 

# feature importance 상위 30개 변수만 사용
# LightGBM 모델의 ROC-AUC 점수: 0.7356
# XGBoost 모델의 ROC-AUC 점수: 0.7278
# CatBoost 모델의 ROC-AUC 점수: 0.7364
# Final 모델의 ROC-AUC 점수: 0.7355

# feature importance ['불임 원인 - 정자 형태', '불임 원인 - 정자 면역학적 요인', '난자 해동 경과일', '불임 원인 - 자궁경부 문제', '불임 원인 - 여성 요인', '난자 채취 경과일'] 제거
# LightGBM 모델의 ROC-AUC 점수: 0.7363
# XGBoost 모델의 ROC-AUC 점수: 0.7299
# CatBoost 모델의 ROC-AUC 점수: 0.7369
# Final 모델의 ROC-AUC 점수: 0.7364  

# kmeans 5개 군집 PCA
# LightGBM 모델의 ROC-AUC 점수: 0.7365
# XGBoost 모델의 ROC-AUC 점수: 0.7297
# CatBoost 모델의 ROC-AUC 점수: 0.7368
# Final 모델의 ROC-AUC 점수: 0.7369

# hyperparameter tuning
# LightGBM 모델의 ROC-AUC 점수: 0.7369
# XGBoost 모델의 ROC-AUC 점수: 0.7368
# CatBoost 모델의 ROC-AUC 점수: 0.7371
# Final 모델의 ROC-AUC 점수: 0.7375

# feature add ADASYN 
# LightGBM 모델의 ROC-AUC 점수: 0.7389
# XGBoost 모델의 ROC-AUC 점수: 0.7386
# CatBoost 모델의 ROC-AUC 점수: 0.7391
# Final 모델의 ROC-AUC 점수: 0.7394

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import  OrdinalEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve

import lightgbm as lgb
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import optuna

numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

train = pd.read_csv('./data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./data/test.csv').drop(columns=['ID'])

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

# 수치형 컬럼들을 0으로 채움
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

# **PCA**로 차원 축소 후 군집화 수행 (군집 수는 3~6 사이로 실험)
pca = PCA(n_components=7, random_state=42)
X_pca = pca.fit_transform(X_train_encoded[numeric_columns])

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans_clusters = kmeans.fit_predict(X_pca)

# 새로운 군집 피처 추가
X_train_encoded['cluster_group'] = kmeans_clusters
X_test_encoded['cluster_group'] = kmeans.predict(pca.transform(X_test_encoded[numeric_columns]))

from imblearn.over_sampling import ADASYN

# ADASYN 적용
adasyn = ADASYN(sampling_strategy=0.5, random_state=42)
X_train_augmented, y_train_augmented = adasyn.fit_resample(X_train_encoded, y)
print("ADASYN 적용 후 데이터 크기:", X_train_augmented.shape, y_train_augmented.shape)

# 새로운 파생 변수 생성
X_train_augmented["시도_대비_이식_비율"] = (
    X_train_augmented["임신 시도 또는 마지막 임신 경과 연수"] / (X_train_augmented["배아 이식 경과일"] + 1)
)
X_train_augmented["난자_활용도"] = (
    X_train_augmented["수집된 신선 난자 수"] / (X_train_augmented["총 생성 배아 수"] + 1)
)

# 테스트 데이터에도 동일하게 적용
X_test_encoded["시도_대비_이식_비율"] = (
    X_test_encoded["임신 시도 또는 마지막 임신 경과 연수"] / (X_test_encoded["배아 이식 경과일"] + 1)
)
X_test_encoded["난자_활용도"] = (
    X_test_encoded["수집된 신선 난자 수"] / (X_test_encoded["총 생성 배아 수"] + 1)
)

print("새로운 변수 추가 완료!")

# 스케일링 적용
scaler = StandardScaler()
X_train_encoded[numeric_columns] = scaler.fit_transform(X_train_encoded[numeric_columns])
X_test_encoded[numeric_columns] = scaler.transform(X_test_encoded[numeric_columns])

print("-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------")
lgb_est = LGBMClassifier(
    class_weight='balanced',
    random_state=42,
    objective='binary',
    n_estimators=1000,
    subsample=0.82,
    colsample_bytree=0.85,
    learning_rate=0.01,
    num_leaves=49,
    max_depth=17,
    device='gpu',  # GPU 사용
    gpu_platform_id=0,
    gpu_device_id=2
)

# LightGBM 모델 학습
lgb_est.fit(X_train_encoded, y)

# LightGBM으로 Feature Importance 확인
feature_importance = pd.DataFrame({
    'feature': X_train_encoded.columns,
    'importance': lgb_est.feature_importances_
}).sort_values(by='importance', ascending=False)

low_importance_features = feature_importance[feature_importance['importance'] <= 10]['feature'].tolist()
print("중요도가 낮은 피처:", low_importance_features)

# 중요도가 낮은 피처 제거
X_train_encoded = X_train_encoded.drop(columns=low_importance_features)
X_test_encoded = X_test_encoded.drop(columns=low_importance_features)

# Train과 Test 데이터의 컬럼이 정확히 일치하는지 확인
X_test_encoded = X_test_encoded[X_train_encoded.columns]


ADASYN 적용 후 데이터 크기: (285026, 68) (285026,)
새로운 변수 추가 완료!
-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 743
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 63
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.89 MB) transferred to GPU in 0.004120 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 

In [9]:
print("-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for LightGBM: {'learning_rate': 0.010682507488382762, 'num_leaves': 49, 'max_depth': 17, 'subsample': 0.8215664737448534, 'colsample_bytree': 0.8497893445033722}
# Best parameters for LightGBM: {'learning_rate': 0.010133602189986561, 'num_leaves': 35, 'max_depth': 9, 'subsample': 0.9394014624416219, 'colsample_bytree': 0.531614077286386}
# Best parameters for LightGBM: {'learning_rate': 0.01404301118940053, 'num_leaves': 33, 'max_depth': 7, 'subsample': 0.9588693733821373, 'colsample_bytree': 0.5190734664368356}
lgb_est = LGBMClassifier(
    class_weight='balanced',
    random_state=42,
    objective='binary',
    n_estimators=1000,
    subsample=0.959,
    colsample_bytree=0.519,
    learning_rate=0.014,
    num_leaves=33,
    max_depth=7,
    device='gpu',  # GPU 사용
    gpu_platform_id=0,
    gpu_device_id=2
)

# LightGBM 모델 학습
lgb_est.fit(X_train_encoded, y)
y_train_pred_lgb = lgb_est.predict_proba(X_train_encoded)[:, 1]
y_test_pred_lgb = lgb_est.predict_proba(X_test_encoded)[:, 1]
roc_auc_lgb = roc_auc_score(y, y_train_pred_lgb)

print("-------------------------- 2. XGBoost 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for XGBoost: {'learning_rate': 0.027828379673739464, 'max_depth': 9, 'subsample': 0.8401352803766647, 'colsample_bytree': 0.817257623227272, 'gamma': 4.054300711621877}
# Best parameters for XGBoost: {'learning_rate': 0.0815351146687336, 'max_depth': 4, 'subsample': 0.9118736980020461, 'colsample_bytree': 0.9163782063094905, 'gamma': 0.9029812557206023}
# Best parameters for XGBoost: {'learning_rate': 0.07820838231358765, 'max_depth': 4, 'subsample': 0.6162659283584726, 'colsample_bytree': 0.9210692408000515, 'gamma': 2.8703958280747597}
xgb_est = XGBClassifier(
    eval_metric='auc',
    random_state=42,
    n_estimators=300,
    learning_rate=0.078,
    max_depth=4,
    subsample=0.616,
    colsample_bytree=0.921,
    gamma=2.870,
    tree_method='hist',  # 새 권장 방식
    device='cuda'        # GPU 사용
)

# XGBoost 모델 학습
xgb_est.fit(X_train_encoded, y)
y_train_pred_xgb = xgb_est.predict_proba(X_train_encoded)[:, 1]
y_test_pred_xgb = xgb_est.predict_proba(X_test_encoded)[:, 1]
roc_auc_xgb = roc_auc_score(y, y_train_pred_xgb)

print("-------------------------- 3. CatBoost 최적 파라미터로 모델 학습 --------------------------")
# Best parameters for CatBoost: {'learning_rate': 0.03222538120048102, 'depth': 6, 'l2_leaf_reg': 1.380048205342132}
# Best parameters for CatBoost: {'learning_rate': 0.023272410542836165, 'depth': 6, 'l2_leaf_reg': 0.02493607891017943}
# Best parameters for CatBoost: {'learning_rate': 0.04136652091082273, 'depth': 5, 'l2_leaf_reg': 4.525500424821638}
catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.0413,
    depth=5,
    l2_leaf_reg=4.5255,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=100,  # 경고 메시지 최소화
    task_type='GPU',
    devices='1'
)

# CatBoost 모델 학습
catboost_model.fit(X_train_encoded, y)
y_train_pred_cat = catboost_model.predict_proba(X_train_encoded)[:, 1]
y_test_pred_cat = catboost_model.predict_proba(X_test_encoded)[:, 1]
roc_auc_cat = roc_auc_score(y, y_train_pred_cat)

# -------------------------- 4. TabNetClassifier 적용 --------------------------
print("-------------------------- 4. TabNetClassifier 학습 --------------------------")

tabnet_clf = TabNetClassifier(
    n_d=256, n_a=256,
    n_steps=5,
    gamma=1.5,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    scheduler_params={"step_size": 10, "gamma": 0.9},
    seed=42
)

tabnet_clf.fit(
    X_train_encoded.values, y.values,
    eval_set=[(X_test_encoded.values, np.zeros(X_test_encoded.shape[0]))],
    eval_metric=['auc'],
    patience=30,
    max_epochs=200
)
y_trian_pred_tabnet = tabnet_clf.predict_proba(X_train_encoded.values)[:, 1]
y_test_pred_tabnet = tabnet_clf.predict_proba(X_test_encoded.values)[:, 1]
roc_auc_tabnet = roc_auc_score(y, y_trian_pred_tabnet)

-------------------------- 1. LightGBM 최적 파라미터로 모델 학습 --------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 741
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 62
[LightGBM] [Info] Using requested OpenCL platform 0 device 2
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (4.89 MB) transferred to GPU in 0.004179 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with posi

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 3.97ms	remaining: 3.96s
100:	total: 306ms	remaining: 2.72s
200:	total: 591ms	remaining: 2.35s
300:	total: 874ms	remaining: 2.03s
400:	total: 1.16s	remaining: 1.73s
500:	total: 1.44s	remaining: 1.43s
600:	total: 1.72s	remaining: 1.14s
700:	total: 2.01s	remaining: 856ms
800:	total: 2.29s	remaining: 569ms
900:	total: 2.57s	remaining: 282ms
999:	total: 2.85s	remaining: 0us
-------------------------- 4. TabNetClassifier 학습 --------------------------


/home/najo/.conda/envs/dip/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


epoch 0  | loss: 0.64753 | val_0_auc: nan     |  0:00:18s


...

epoch 28 | loss: 0.48828 | val_0_auc: nan     |  0:08:54s


/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


epoch 29 | loss: 0.48807 | val_0_auc: nan     |  0:09:11s


/home/najo/.conda/envs/dip/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/najo/.conda/envs/dip/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 30 | loss: 0.48776 | val_0_auc: nan     |  0:09:31s

Early stopping occurred at epoch 30 with best_epoch = 0 and best_val_0_auc = -inf


In [10]:
# -------------------------- 4. Weighted Ensemble --------------------------
print("-------------------------- 4. Weighted Ensemble 최적 가중치 탐색 --------------------------")

def objective_weights(trial):
    w1 = trial.suggest_float('w1', 0, 1)
    w2 = trial.suggest_float('w2', 0, 1 - w1)
    remaining = max(1 - w1 - w2, 0)  # 음수 방지
    w3 = trial.suggest_float('w3', 0, remaining)
    w4 = max(remaining - w3, 0)  # 마지막 가중치는 남은 값으로 설정

    final_pred_proba = (
        w1 * y_train_pred_lgb +
        w2 * y_train_pred_xgb +
        w3 * y_trian_pred_tabnet +
        w4 * y_train_pred_cat
    )
    return roc_auc_score(y, final_pred_proba)

study_weights = optuna.create_study(direction='maximize')
study_weights.optimize(objective_weights, n_trials=1000)
best_weights = study_weights.best_params

print(f"최적 가중치: LightGBM={best_weights['w1']:.2f}, XGBoost={best_weights['w2']:.2f}, TabNetClassifier={best_weights['w3']:.2f}, CatBoost={1 - best_weights['w1'] - best_weights['w2'] - best_weights['w3']:.2f}")
# 최적 가중치: LightGBM=0.21, XGBoost=0.36, TabNetClassifier=0.28, CatBoost=0.15

# 최종 앙상블 적용
final_pred_proba = (
    best_weights['w1'] * y_train_pred_lgb +
    best_weights['w2'] * y_train_pred_xgb +
    best_weights['w3'] * y_trian_pred_tabnet +
    (1 - best_weights['w1'] - best_weights['w2'] - best_weights['w3']) * y_train_pred_cat
)
roc_auc_final = roc_auc_score(y, final_pred_proba)
print(f"Train Final 앙상블 모델의 ROC-AUC 점수: {roc_auc_final:.4f}")

[I 2025-02-13 03:13:53,504] A new study created in memory with name: no-name-5a5e8f6d-cb17-46e4-8f33-7e5a402a10d5
[I 2025-02-13 03:13:53,618] Trial 0 finished with value: 0.7502698429228278 and parameters: {'w1': 0.5583057746083957, 'w2': 0.14658264852425135, 'w3': 0.26799940835229014}. Best is trial 0 with value: 0.7502698429228278.


-------------------------- 4. Weighted Ensemble 최적 가중치 탐색 --------------------------


[I 2025-02-13 03:13:53,729] Trial 1 finished with value: 0.7495777566344203 and parameters: {'w1': 0.27627331766477037, 'w2': 0.3117080491597264, 'w3': 0.13044790277716442}. Best is trial 0 with value: 0.7502698429228278.
[I 2025-02-13 03:13:53,839] Trial 2 finished with value: 0.7508846946782108 and parameters: {'w1': 0.5793542312085143, 'w2': 0.24060506674378854, 'w3': 0.017138130256895888}. Best is trial 2 with value: 0.7508846946782108.
[I 2025-02-13 03:13:53,951] Trial 3 finished with value: 0.7500059256801186 and parameters: {'w1': 0.45571285674941786, 'w2': 0.04191390674753844, 'w3': 0.3127511982019409}. Best is trial 2 with value: 0.7508846946782108.
[I 2025-02-13 03:13:54,063] Trial 4 finished with value: 0.7485125453275614 and parameters: {'w1': 0.3024110929203043, 'w2': 0.06778782158023614, 'w3': 0.5106098332588606}. Best is trial 2 with value: 0.7508846946782108.
[I 2025-02-13 03:13:54,172] Trial 5 finished with value: 0.7498708677770867 and parameters: {'w1': 0.32895097623

최적 가중치: LightGBM=1.00, XGBoost=0.00, TabNetClassifier=0.00, CatBoost=0.00
Train Final 앙상블 모델의 ROC-AUC 점수: 0.7521


In [13]:
print(f"Train LightGBM 모델의 ROC-AUC 점수: {roc_auc_lgb:.4f}")
print(f"Train XGBoost 모델의 ROC-AUC 점수: {roc_auc_xgb:.4f}")
print(f"Train CatBoost 모델의 ROC-AUC 점수: {roc_auc_cat:.4f}")
print(f"Train TabNetClassifier 모델의 ROC-AUC 점수: {roc_auc_tabnet:.4f}")

# 최적 가중치: LightGBM=0.21, XGBoost=0.36, TabNetClassifier=0.28, CatBoost=0.15
# 최적 가중치: LightGBM=0.00, XGBoost=0.23, TabNetClassifier=0.22, CatBoost=0.55

# 최종 앙상블 적용
final_train_pred_proba = (
    0 * y_train_pred_lgb +
    0.23 * y_train_pred_xgb +
    0.22 * y_trian_pred_tabnet +
    0.55 * y_train_pred_cat
)

final_test_pred_proba = (
    0.5 * y_test_pred_lgb +
    0 * y_test_pred_xgb +
    0 * y_test_pred_tabnet +
    0.5 * y_test_pred_cat
)

roc_auc_final = roc_auc_score(y, final_train_pred_proba)
print(f"Train Final 앙상블 모델의 ROC-AUC 점수: {roc_auc_final:.4f}")

# 제출 파일 생성
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['probability'] = final_test_pred_proba
sample_submission.to_csv('./submit/final_submit.csv', index=False)

print("최종 제출 파일 'stacking_ensemble_submit.csv' 생성 완료!")

Train LightGBM 모델의 ROC-AUC 점수: 0.7521
Train XGBoost 모델의 ROC-AUC 점수: 0.7471
Train CatBoost 모델의 ROC-AUC 점수: 0.7490
Train TabNetClassifier 모델의 ROC-AUC 점수: 0.7436
Train Final 앙상블 모델의 ROC-AUC 점수: 0.7482


최종 제출 파일 'stacking_ensemble_submit.csv' 생성 완료!
